In [167]:
import pandas as pd
import warnings





In [173]:
data = pd.read_csv("CLV Retail data Set.csv", encoding='ISO-8859-1')
data.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [96]:
# Some small data wrangling task
data['total_revenue'] = data['Quantity'] * data['UnitPrice']
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data = data[data['Quantity'] > 0]

In [68]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


# creating grouping around the customer ID and aggregate the metrics

In [189]:
# Convert InvoiceDate to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Then proceed with your logic
max_date = data['InvoiceDate'].max()

data = data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (max_date - x.min()).days,
    'InvoiceNo': lambda x: len(x),
    'Quantity': lambda x: x.sum(),
    'UnitPrice': lambda x: (x * data.loc[x.index, 'Quantity']).sum()
})

data.rename(columns={'UnitPrice': 'total_revenue'}, inplace=True)


In [193]:
data.head()

,InvoiceDate,InvoiceNo,Quantity,total_revenue
CustomerID,,,,
12346.0,325,2,0,0.00
12347.0,366,182,2458,4310.00
12348.0,357,31,2341,1797.24
12349.0,18,73,631,1757.55
12350.0,309,17,197,334.40


#  putting this all together in a function

In [98]:
# Lets put this all together in a function
def customer_model(data):
    # Convert InvoiceDate to datetime if not already
    data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

    # Get the maximum date
    max_date = data['InvoiceDate'].max()

    # Add a new column for total revenue
    data['total_revenue'] = data['Quantity'] * data['UnitPrice']

    # Grouping by CustomerID and aggregating
    data = data.groupby('CustomerID').agg({
        'InvoiceDate': lambda x: (max_date - x.min()).days,
        'InvoiceNo': 'nunique',
        'Quantity': 'sum',
        'total_revenue': 'sum'
    })

    return data

data = customer_model(data)
data.head()


,InvoiceDate,InvoiceNo,Quantity,total_revenue
CustomerID,,,,
12346.0,325,1,74215,77183.60
12347.0,366,7,2458,4310.00
12348.0,357,4,2341,1797.24
12349.0,18,1,631,1757.55
12350.0,309,1,197,334.40


In [100]:
# Change the name of the columns and ensure that we don't have 0 quantity
data.columns = ['age', 'num_transactions', 'quantity', 'total_revenue']
data = data[data['quantity'] > 0]
data.head()


,age,num_transactions,quantity,total_revenue
CustomerID,,,,
12346.0,325,1,74215,77183.60
12347.0,366,7,2458,4310.00
12348.0,357,4,2341,1797.24
12349.0,18,1,631,1757.55
12350.0,309,1,197,334.40


In [102]:
data['AOV'] =data['total_revenue']/data['num_transactions']
data

,age,num_transactions,quantity,total_revenue,AOV
CustomerID,,,,,
12346.0,325,1,74215,77183.60,77183.600000
12347.0,366,7,2458,4310.00,615.714286
12348.0,357,4,2341,1797.24,449.310000
12349.0,18,1,631,1757.55,1757.550000
12350.0,309,1,197,334.40,334.400000
...,...,...,...,...,...
18280.0,277,1,45,180.60,180.600000
18281.0,180,1,54,80.82,80.820000
18282.0,125,2,103,178.05,89.025000


In [110]:

purchase_freq = sum(data['num_transactions'])/len(data)
purchase_freq

4.271952062687255

In [120]:
data.shape

(4339, 6)

In [112]:

repeat_rate = data[data['num_transactions']>1].shape[0]/data.shape[0]
repeat_rate

0.6556810324959668

In [116]:
churn_rate = 1-repeat_rate
churn_rate

0.34431896750403324

In [118]:
data['profit_margin'] = data['total_revenue']*.10

In [124]:
data.head()

,age,num_transactions,quantity,total_revenue,AOV,profit_margin
CustomerID,,,,,,
12346.0,325,1,74215,77183.60,77183.600000,7718.360
12347.0,366,7,2458,4310.00,615.714286,431.000
12348.0,357,4,2341,1797.24,449.310000,179.724
12349.0,18,1,631,1757.55,1757.550000,175.755
12350.0,309,1,197,334.40,334.400000,33.440


# putting equation together 

In [129]:
data['CLTV'] = ((data['AOV'] * purchase_freq) / churn_rate) * 0.10
data


,age,num_transactions,quantity,total_revenue,AOV,profit_margin,CLTV
CustomerID,,,,,,,
12346.0,325,1,74215,77183.60,77183.600000,7718.360,95761.392878
12347.0,366,7,2458,4310.00,615.714286,431.000,763.914324
12348.0,357,4,2341,1797.24,449.310000,179.724,557.457173
12349.0,18,1,631,1757.55,1757.550000,175.755,2180.585462
12350.0,309,1,197,334.40,334.400000,33.440,414.888782
...,...,...,...,...,...,...,...
18280.0,277,1,45,180.60,180.600000,18.060,224.069719
18281.0,180,1,54,80.82,80.820000,8.082,100.273060
18282.0,125,2,103,178.05,89.025000,17.805,110.452972


In [131]:
#import seaborn
import seaborn as sns
print("The median CLTV is", data['CLTV'].median())
print("The average CLTV is", data['CLTV'].mean())


The median CLTV is 364.3800428380187
The average CLTV is 519.9108564852671


In [134]:
data.sort_values(by='CLTV').reset_index()


,CustomerID,age,num_transactions,quantity,total_revenue,AOV,profit_margin,CLTV
0,13256.0,13,1,12540,0.00,0.000000,0.000,0.000000
1,16454.0,63,2,3,6.90,3.450000,0.690,4.280402
2,16738.0,297,1,3,3.75,3.750000,0.375,4.652610
3,14792.0,63,1,2,6.20,6.200000,0.620,7.692316
4,17816.0,364,4,30,36.56,9.140000,3.656,11.339963
...,...,...,...,...,...,...,...,...
4334,12357.0,32,1,2708,6207.67,6207.670000,620.767,7701.832070
4335,15098.0,181,3,121,39916.50,13305.500000,3991.650,16508.082195
4336,15749.0,331,3,18028,44534.30,14844.766667,4453.430,18417.844373
4337,12346.0,325,1,74215,77183.60,77183.600000,7718.360,95761.392878


In [138]:
# Remove display limits
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [142]:
data.sort_values(by='CLTV', ascending=False).reset_index(drop=True)


,age,num_transactions,quantity,total_revenue,AOV,profit_margin,CLTV
0,205,2,80997,168472.500,84236.250000,16847.2500,104511.588353
1,325,1,74215,77183.600,77183.600000,7718.3600,95761.392878
2,331,3,18028,44534.300,14844.766667,4453.4300,18417.844373
3,181,3,121,39916.500,13305.500000,3991.6500,16508.082195
4,32,1,2708,6207.670,6207.670000,620.7670,7701.832070
5,337,21,77670,124914.530,5948.310952,12491.4530,7380.046306
6,210,2,4283,9864.260,4932.130000,986.4260,6119.274544
7,113,1,3028,4873.810,4873.810000,487.3810,6046.917146
8,80,1,2262,4366.780,4366.780000,436.6780,5417.846993
9,366,60,64124,259657.300,4327.621667,25965.7300,5369.263401
